# Introduction

This document serves to provide an orientation to differnet OpenSSL commands.

## Computing Environment

Some basic information about the environment in which the Nobeook is executing.

In [ ]:
%%bash
# This script is used to print the system information of the current machine
# It is useful to know the system information when debugging issues


echo "Kernel: `uname -a`"
echo "Operating System:"
if [ -f /etc/os-release ]; then
    cat /etc/os-release | awk '{print "\t" $0}'
fi;
if uname -a | grep -iq "darwin"; then
    sw_vers | awk '{print "\t" $0}'
fi;

echo "OpenSSL: `openssl version`"
echo "Python: `python --version`"

Kernel: Darwin USWSA220038.local 24.1.0 Darwin Kernel Version 24.1.0: Thu Oct 10 21:03:15 PDT 2024; root:xnu-11215.41.3~2/RELEASE_ARM64_T6000 arm64
Operating System:
	ProductName:		macOS
	ProductVersion:		15.1.1
	BuildVersion:		24B91
OpenSSL: OpenSSL 3.4.0 22 Oct 2024 (Library: OpenSSL 3.4.0 22 Oct 2024)
Python: Python 3.11.6


## Available RSA Key Sizes

Below are the exponents associated with given RSA key lengths. 

In [9]:
#python
# Some possible RSA key lenths.
for exponent in range(7,16):
    print("2^{0:3d} = {1:5d} bits".format(exponent, 2**exponent))

2^  7 =   128 bits
2^  8 =   256 bits
2^  9 =   512 bits
2^ 10 =  1024 bits
2^ 11 =  2048 bits
2^ 12 =  4096 bits
2^ 13 =  8192 bits
2^ 14 = 16384 bits
2^ 15 = 32768 bits


Note, that 3072-bit (2048+1024) keys are not referenced in this list.  It is likely that 3072-bit is trying to strike a cost/performance balance between weak 2048-bit and high computation cost of 4096-bit RSA keys.  So where to 3072-bit RSA keys come from?

In [19]:
# RSA key sizes are often represented as exponents to 2.
print(f"2048-bit RSA keys are derived from 2^11 = {2**11}.")
print(f"Where does 3072-bit RSA keys come from?")
print(f"- 3072-bit RSA keys might be 2^11.584962501 = {2**11.584962501}.")
print(f"- Or simply 2048*1.5 = {2048*1.5}.")

2048-bit RSA keys are derived from 2^11 = 2048.
Where does 3072-bit RSA keys come from?
- 3072-bit RSA keys might be 2^11.584962501 = 3072.000000593755.
- Or simply 2048*1.5 = 3072.0.


## Available Elliptic Curves

Some resources on about elliptic curves applied to TLS protocol.   
* [RFC 8422, Section 5.1.1 Supported Elliptic Curves Extensions](https://www.rfc-editor.org/rfc/rfc8422.html#section-5.1.1) namely `secp256r1`, `secp384r1`, `secp521r1` and a few others.
* [AWS Security Blog](https://aws.amazon.com/blogs/security/) discusses [How to evaluate and use ECDSA certificates in AWS Certificate Manager](https://aws.amazon.com/blogs/security/how-to-evaluate-and-use-ecdsa-certificates-in-aws-certificate-manager/).  It is acknowledged this article discusses ECDSA and not EC. It was just an simple read on EC curve options.

For the sake of simplicity and scoping, will focus on `prime256v1` and `secp384r1` curves only.  The tool `openssl` offers support for several different elliptic curves as noted below.

In [20]:
#!/bin/bash
# What are the EC Curves supported by OpenSSL?
!openssl ecparam -list_curves

  secp112r1 : SECG/WTLS curve over a 112 bit prime field
  secp112r2 : SECG curve over a 112 bit prime field
  secp128r1 : SECG curve over a 128 bit prime field
  secp128r2 : SECG curve over a 128 bit prime field
  secp160k1 : SECG curve over a 160 bit prime field
  secp160r1 : SECG curve over a 160 bit prime field
  secp160r2 : SECG/WTLS curve over a 160 bit prime field
  secp192k1 : SECG curve over a 192 bit prime field
  secp224k1 : SECG curve over a 224 bit prime field
  secp224r1 : NIST/SECG curve over a 224 bit prime field
  secp256k1 : SECG curve over a 256 bit prime field
  secp384r1 : NIST/SECG curve over a 384 bit prime field
  secp521r1 : NIST/SECG curve over a 521 bit prime field
  prime192v1: NIST/X9.62/SECG curve over a 192 bit prime field
  prime192v2: X9.62 curve over a 192 bit prime field
  prime192v3: X9.62 curve over a 192 bit prime field
  prime239v1: X9.62 curve over a 239 bit prime field
  prime239v2: X9.62 curve over a 239 bit prime field
  prime239v3: X9.62 curv

# X.509 Key Types

The goal of this section is to demonstrate the differences between message digests (or thumbprints) between two different (but related) data structures with X.509 certificates.  
* Thumbprint of the signed certificate.
* Thumbprint of the cryptographic public key.

In the following sections we use RSA and ECC X.509 key types to perform the following tasks.
1. Create an single private key.
2. Derive from this private key the corresponding private key.
3. Generate multiple certificate signing requesest based on the single private key.
4. Self-sign each signing request with the private key.
5. We then extract the SubjectPublicKeyInfo from each signing request and self-signed certificate.
6. Finally, we compare the SubjectPublicKeyeInfo across all the various exports.

File names follow a mixed combination of hyphens (`-`) to separate words within a given field (e.g. `private-key`) and underscores (`_`) to separate fields (e.g. `rsa_private-key`).  


## RSA

Resources
1. [openssl-asn1parse](https://docs.openssl.org/master/man1/openssl-asn1parse/) processing ASN.1 files
2. [openssl-pkey](https://docs.openssl.org/master/man1/openssl-pkey/) interacting with private keys
3. [openssl-req](https://docs.openssl.org/master/man1/openssl-req/) processing signing-requests
4. [openssl-rsa](https://docs.openssl.org/master/man1/openssl-rsa/) generating RSA private keys
5. [openssl-x509](https://docs.openssl.org/master/man1/openssl-x509/) processing X.509 certificates

### Generate RSA Private Key

For illustration purposes, let us create a single RSA private key. 

> Warning: Please do not use these keys in any production or live environment as they are intended for demonstration purposes only.  As a _demo_, you have no control over who may have viewed or obtain a copy of this key material making these private keys potentially _public_.

In [21]:
%%bash
# preamble
FOLDER="data/poc"
mkdir -p "${FOLDER}"
RSA_PRIVATE_KEY_PEM="${FOLDER}/rsa_private-key.pem"
RSA_PRIVATE_KEY_DER="${FOLDER}/rsa_private-key.der"
KEYLENGTH="2048" # Typical sizes: 2048, 3072, 4096`

# validate environment
echo "Environment:"
echo "  Working Folder: '${FOLDER}'"
echo "  Private Key PEM: '${RSA_PRIVATE_KEY_PEM}'"
echo "  Private Key DER: '${RSA_PRIVATE_KEY_DER}'"
echo "  Key Length: '${KEYLENGTH}'"
echo ""

# execute
echo "Generating RSA Private Key:"

# create the private key in PEM form.
openssl genrsa -out "${RSA_PRIVATE_KEY_PEM}" "${KEYLENGTH}"

# convert private key from PEM (ASCII) to DER (binary)
openssl rsa -inform PEM -in "${RSA_PRIVATE_KEY_PEM}" -outform DER -out "${RSA_PRIVATE_KEY_DER}"
echo ""

# valdiate result
echo "Files:"
ls -l -go "${FOLDER}"/rsa_private-key.*
echo ""

echo "File Types:"
file "${RSA_PRIVATE_KEY_PEM}" "${RSA_PRIVATE_KEY_DER}"
echo ""

echo "File Digests (SHA256):"
sha256sum "${FOLDER}"/rsa_private-key.*

Environment:
  Working Folder: 'data/poc'
  Private Key PEM: 'data/poc/rsa_private-key.pem'
  Private Key DER: 'data/poc/rsa_private-key.der'
  Key Length: '2048'

Generating RSA Private Key:


writing RSA key



Files:
-rw------- 1 1216 Nov 21 17:26 data/poc/rsa_private-key.der
-rw------- 1 1704 Nov 21 17:26 data/poc/rsa_private-key.pem

File Types:
data/poc/rsa_private-key.pem: OpenSSH private key (no password)
data/poc/rsa_private-key.der: data

File Digests (SHA256):
2e344928b2d1e38cd115fd93f96ad3e65666a0a860fa95bafb5d17d2279d5355  data/poc/rsa_private-key.der
bb63f50cb1817ff667eb5f92776ac35b6d444dd02c53508b6d5818fceef0595b  data/poc/rsa_private-key.pem


### Derive the RSA Public Key

The raw private key material itself does not contain the public key.  However we can derive the cryptographic public key material from the private key using one of the `openssl` command options.  
1. `openssl rsa -pubout ...`
2. `openssl pkey -pubout ...`

In [22]:
%%bash
# preamble
FOLDER="data/poc"
mkdir -p "${FOLDER}"
RSA_PRIVATE_KEY_PEM="${FOLDER}/rsa_private-key.pem"
RSA_PRIVATE_KEY_DER="${FOLDER}/rsa_private-key.der"


# validate environment
echo "Environment:"
echo "  RSA Private Key PEM: '${RSA_PRIVATE_KEY_PEM}'"
echo "  RSA Private Key DER: '${RSA_PRIVATE_KEY_DER}'"
echo ""

# execute
echo "Deriving the RSA Public Key using the corresponding Private Key (rsa):"
openssl rsa -pubout -inform PEM -in "${RSA_PRIVATE_KEY_PEM}" \
    -outform PEM -out "${FOLDER}/rsa_public-key-info_via_private-key_rsa.pem"
openssl rsa -pubout -inform DER -in "${RSA_PRIVATE_KEY_DER}" \
    -outform DER -out "${FOLDER}/rsa_public-key-info_via_private-key_rsa.der"
echo ""

echo "Deriving the RSA Public Key using the corresponding Private Key (rsa):"
openssl pkey -pubout -inform PEM -in "${RSA_PRIVATE_KEY_PEM}" \
    -outform PEM -out "${FOLDER}/rsa_public-key-info_via_private-key_pkey.pem"
openssl pkey -pubout -inform DER -in "${RSA_PRIVATE_KEY_DER}" \
    -outform DER -out "${FOLDER}/rsa_public-key-info_via_private-key_pkey.der"
echo ""

# validate results
echo "RSA Files:"
ls -l -go "${FOLDER}"/rsa_public-key-info_via_private-key*
echo ""

echo "RSA File Types:"
file "${FOLDER}"/rsa_public-key-info_via_private-key*
echo ""

echo "RSA File Digests (SHA256):"
sha256sum "${FOLDER}"/rsa_public-key-info_via_private-key*.der
echo "---"
sha256sum "${FOLDER}"/rsa_public-key-info_via_private-key*.pem

Environment:
  RSA Private Key PEM: 'data/poc/rsa_private-key.pem'
  RSA Private Key DER: 'data/poc/rsa_private-key.der'

Deriving the RSA Public Key using the corresponding Private Key (rsa):


writing RSA key
writing RSA key



Deriving the RSA Public Key using the corresponding Private Key (rsa):

RSA Files:
-rw-rw-r-- 1 294 Nov 21 17:26 data/poc/rsa_public-key-info_via_private-key_pkey.der
-rw-rw-r-- 1 451 Nov 21 17:26 data/poc/rsa_public-key-info_via_private-key_pkey.pem
-rw-rw-r-- 1 294 Nov 21 17:26 data/poc/rsa_public-key-info_via_private-key_rsa.der
-rw-rw-r-- 1 451 Nov 21 17:26 data/poc/rsa_public-key-info_via_private-key_rsa.pem

RSA File Types:
data/poc/rsa_public-key-info_via_private-key_pkey.der: data
data/poc/rsa_public-key-info_via_private-key_pkey.pem: ASCII text
data/poc/rsa_public-key-info_via_private-key_rsa.der:  data
data/poc/rsa_public-key-info_via_private-key_rsa.pem:  ASCII text

RSA File Digests (SHA256):
7739a822aa5745263aa46d9aa663b53b3c011f4ac552b548d80b4d7a669ccebd  data/poc/rsa_public-key-info_via_private-key_pkey.der
7739a822aa5745263aa46d9aa663b53b3c011f4ac552b548d80b4d7a669ccebd  data/poc/rsa_public-key-info_via_private-key_rsa.der
---
4835c613f9e0f4ad54e97251c068f1a9f390d7541a

### Generate RSA Signing Requests

Our goal is to create multiple unique signed public certificates each based on the single private key we just created.  To begin, we must create a set of _certificate signing requests_, each provided with a unique name.  

To uniquely identify each signing request (and later the certificate) we apply a unique ID to the request `CommonName` field of the `Subject`.

In [23]:
%%bash
FOLDER="data/poc"
RSA_PRIVATE_KEY_PEM="${FOLDER}/rsa_private-key.pem"
MAX_SIGNING_REQUESTS=5

echo "Environment:"
echo "  Signing Request Count: ${MAX_SIGNING_REQUESTS}"
echo ""

for ((i=1; i<="${MAX_SIGNING_REQUESTS}"; i++)); do
    GUID=$(uuidgen)
    RSA_SIGNING_REQUEST_PEM="${FOLDER}/rsa_signing-request_${GUID}.pem"
    RSA_SIGNING_REQUEST_DER="${FOLDER}/rsa_signing-request_${GUID}.der"
    
    # creates PEM signing request
    echo "Creating PEM: ${RSA_SIGNING_REQUEST_PEM}"
    openssl req -new -sha256 -inform pem -key "${RSA_PRIVATE_KEY_PEM}" \
      -outform pem -out "${RSA_SIGNING_REQUEST_PEM}" -subj "/O=WARNING\: PRIVATE KEY MADE PUBLIC/CN=${GUID}"

    # converts PEM to DER signing request
    echo "Creating DER: ${RSA_SIGNING_REQUEST_DER}"
    openssl req -inform PEM -in "${RSA_SIGNING_REQUEST_PEM}" \
      -outform DER -out "${RSA_SIGNING_REQUEST_DER}"

done
echo ""

echo "RSA Signing Requests:"
ls -l -go "${FOLDER}"/rsa_signing-request*
echo ""

echo "RSA File Types:"
file "${FOLDER}"/rsa_signing-request*
echo ""

echo "RSA Signing Request Digests (SHA256):"
sha256sum "${FOLDER}"/rsa_signing-request*.der
echo "---"
sha256sum "${FOLDER}"/rsa_signing-request*.pem
echo ""


Environment:
  Signing Request Count: 5

Creating PEM: data/poc/rsa_signing-request_1f181c3b-551e-4e63-85ac-0724124888ed.pem
Creating DER: data/poc/rsa_signing-request_1f181c3b-551e-4e63-85ac-0724124888ed.der
Creating PEM: data/poc/rsa_signing-request_311affb2-a2e2-47de-88ee-5eb5cd0b5f59.pem
Creating DER: data/poc/rsa_signing-request_311affb2-a2e2-47de-88ee-5eb5cd0b5f59.der
Creating PEM: data/poc/rsa_signing-request_1eb88898-fb38-4a66-81a4-16e11175edb5.pem
Creating DER: data/poc/rsa_signing-request_1eb88898-fb38-4a66-81a4-16e11175edb5.der
Creating PEM: data/poc/rsa_signing-request_3550c8ce-7c71-4570-8358-672787735503.pem
Creating DER: data/poc/rsa_signing-request_3550c8ce-7c71-4570-8358-672787735503.der
Creating PEM: data/poc/rsa_signing-request_1beba118-c83f-4095-883d-3f19af5d58ca.pem
Creating DER: data/poc/rsa_signing-request_1beba118-c83f-4095-883d-3f19af5d58ca.der

RSA Signing Requests:
-rw-rw-r-- 1 675 Nov 21 17:26 data/poc/rsa_signing-request_1beba118-c83f-4095-883d-3f19af5d58ca.

### Self-Sign RSA Signing Requests

Using the single private key, we can self-sign each of the signing requests to create a completed signed certificate for analysis.  For added variability with our certificates, each has a unique validity duration.

In [24]:
%%bash
FOLDER="data/poc"
RSA_PRIVATE_KEY_PEM="${FOLDER}/rsa_private-key.pem"
VALIDITY_DURATION=30
VALIDITY_INCREMENT=10

for RSA_SIGNING_REQUEST_PEM in "${FOLDER}"/rsa_signing-request_*.pem; do
    VALIDITY_DURATION=$((VALIDITY_DURATION+VALIDITY_INCREMENT))
    GUID=$(openssl req -noout -in "${RSA_SIGNING_REQUEST_PEM}" -subject | awk -F 'CN=' '{print $2}')
    RSA_SELF_SIGNED_PUBLIC_KEY_PEM="${FOLDER}/rsa_self-signed-public-key_${GUID}.pem" 
    RSA_SELF_SIGNED_PUBLIC_KEY_DER="${FOLDER}/rsa_self-signed-public-key_${GUID}.der"  

    # creating PEM signed certificate
    echo "Creating Self-Signed PEM: ${RSA_SELF_SIGNED_PUBLIC_KEY_PEM}"
    openssl x509 -req -sha256 -days ${VALIDITY_DURATION} \
        -keyform PEM -signkey "${RSA_PRIVATE_KEY_PEM}" \
        -inform PEM -in "${RSA_SIGNING_REQUEST_PEM}" \
        -outform PEM -out "${RSA_SELF_SIGNED_PUBLIC_KEY_PEM}"

    # converting PEM to DER
    echo "Converting PEM to DER: ${RSA_SELF_SIGNED_PUBLIC_KEY_DER}"
    openssl x509 -inform PEM -in "${RSA_SELF_SIGNED_PUBLIC_KEY_PEM}" \
        -outform DER -out "${RSA_SELF_SIGNED_PUBLIC_KEY_DER}"
    echo ""
done
echo ""

echo "RSA Self-Signed Certificates:"
ls -l -go "${FOLDER}"/rsa_self-signed-public-key*
echo ""

echo "RSA Self-Signed Certificate File Types:"
file "${FOLDER}"/rsa_self-signed-public-key*
echo ""

echo "RSA Self-Signed Certificate Digests (SHA256):"
sha256sum "${FOLDER}"/rsa_self-signed-public-key*.der
echo "---"
sha256sum "${FOLDER}"/rsa_self-signed-public-key*.pem
echo ""


Creating Self-Signed PEM: data/poc/rsa_self-signed-public-key_1beba118-c83f-4095-883d-3f19af5d58ca.pem


Certificate request self-signature ok
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=1beba118-c83f-4095-883d-3f19af5d58ca


Converting PEM to DER: data/poc/rsa_self-signed-public-key_1beba118-c83f-4095-883d-3f19af5d58ca.der

Creating Self-Signed PEM: data/poc/rsa_self-signed-public-key_1eb88898-fb38-4a66-81a4-16e11175edb5.pem


Certificate request self-signature ok
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=1eb88898-fb38-4a66-81a4-16e11175edb5


Converting PEM to DER: data/poc/rsa_self-signed-public-key_1eb88898-fb38-4a66-81a4-16e11175edb5.der

Creating Self-Signed PEM: data/poc/rsa_self-signed-public-key_1f181c3b-551e-4e63-85ac-0724124888ed.pem


...

Converting PEM to DER: data/poc/rsa_self-signed-public-key_1f181c3b-551e-4e63-85ac-0724124888ed.der

Creating Self-Signed PEM: data/poc/rsa_self-signed-public-key_311affb2-a2e2-47de-88ee-5eb5cd0b5f59.pem


Certificate request self-signature ok
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=311affb2-a2e2-47de-88ee-5eb5cd0b5f59


Converting PEM to DER: data/poc/rsa_self-signed-public-key_311affb2-a2e2-47de-88ee-5eb5cd0b5f59.der

Creating Self-Signed PEM: data/poc/rsa_self-signed-public-key_3550c8ce-7c71-4570-8358-672787735503.pem


Certificate request self-signature ok
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=3550c8ce-7c71-4570-8358-672787735503


Converting PEM to DER: data/poc/rsa_self-signed-public-key_3550c8ce-7c71-4570-8358-672787735503.der


RSA Self-Signed Certificates:
-rw-rw-r-- 1  871 Nov 21 17:26 data/poc/rsa_self-signed-public-key_1beba118-c83f-4095-883d-3f19af5d58ca.der
-rw-rw-r-- 1 1237 Nov 21 17:26 data/poc/rsa_self-signed-public-key_1beba118-c83f-4095-883d-3f19af5d58ca.pem
-rw-rw-r-- 1  871 Nov 21 17:26 data/poc/rsa_self-signed-public-key_1eb88898-fb38-4a66-81a4-16e11175edb5.der
-rw-rw-r-- 1 1237 Nov 21 17:26 data/poc/rsa_self-signed-public-key_1eb88898-fb38-4a66-81a4-16e11175edb5.pem
-rw-rw-r-- 1  871 Nov 21 17:26 data/poc/rsa_self-signed-public-key_1f181c3b-551e-4e63-85ac-0724124888ed.der
-rw-rw-r-- 1 1237 Nov 21 17:26 data/poc/rsa_self-signed-public-key_1f181c3b-551e-4e63-85ac-0724124888ed.pem
-rw-rw-r-- 1  871 Nov 21 17:26 data/poc/rsa_self-signed-public-key_311affb2-a2e2-47de-88ee-5eb5cd0b5f59.der
-rw-rw-r-- 1 1237 Nov 21 17:26 data/poc/rsa_self-signed-public-key_311affb2-a2e2-47de-88ee-5eb5cd0b5f59.pem
-rw-

### Display Certificate Details (Briefly)

Confirming our certificates have unique names and validity periods.

In [25]:
%%bash
FOLDER="data/poc"

# from signed certificates
echo "Basic Certificate Details:"
for RSA_SELF_SIGNED_PUBLIC_KEY_PEM in "${FOLDER}"/rsa_self-signed-public-key*.pem; do
    echo "--------------------------------------------"
    openssl x509 -noout -in "${RSA_SELF_SIGNED_PUBLIC_KEY_PEM}" \
        -subject -serial -startdate -enddate
done
echo ""

Basic Certificate Details:
--------------------------------------------
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=1beba118-c83f-4095-883d-3f19af5d58ca
serial=49451EEB486BD35584EC80DB37968AB52D29E2E7
notBefore=Nov 21 22:26:37 2024 GMT
notAfter=Dec 31 22:26:37 2024 GMT
--------------------------------------------
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=1eb88898-fb38-4a66-81a4-16e11175edb5
serial=1B06FDE520C5DF414F67922FCAC7C08FDEC65C67
notBefore=Nov 21 22:26:37 2024 GMT
notAfter=Jan 10 22:26:37 2025 GMT
--------------------------------------------
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=1f181c3b-551e-4e63-85ac-0724124888ed
serial=30334658186F4A9C2354D237B4A27E51D6CD5897
notBefore=Nov 21 22:26:37 2024 GMT
notAfter=Jan 20 22:26:37 2025 GMT
--------------------------------------------
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=311affb2-a2e2-47de-88ee-5eb5cd0b5f59
serial=68CC49632C46272C8BFA959B3286B820687B21C1
notBefore=Nov 21 22:26:37 2024 GMT
notAfter=Jan 30 22:26:3

### Extract RSA SubjectPublicKeyInfo

We already derived the raw public key material earlier by accessing the private key for this information.  Here, we'll shift our focus to both the signing requests and self-signed certificates for analysis.

First, we'll access the signing requests to extract the `SubjectPublicKeyInfo` data and convert it to DER (binary) format.  Then we'll access each self-signed certificate to perform a similar action.

> Note: when using the `-pubkey` option, the output will often contain two sections, in ASCII format, defining the `PUBLIC KEY` and `CERTIFICATE REQUEST`.  We use `sed` to parse out only the `PUBLIC KEY` portion of this output.

In [26]:
%%bash
FOLDER="data/poc"

echo "Extracting the SubjectPublicKeyInfo from RSA Signing Requests:"
echo "--------------------------------------------"
# from signing request
for RSA_SIGNING_REQUEST_PEM in "${FOLDER}"/rsa_signing-request*.pem; do
    GUID=$(openssl req -noout -in "${RSA_SIGNING_REQUEST_PEM}" -subject | awk -F 'CN=' '{print $2}')
    RSA_PUBLIC_KEY_INFO_SIGNING_REQUEST_PEM="${FOLDER}/rsa_public-key-info_via_signing-request_${GUID}.pem" 
    RSA_PUBLIC_KEY_INFO_SIGNING_REQUEST_DER="${FOLDER}/rsa_public-key-info_via_signing-request_${GUID}.der"  

    # creating PEM signed certificate
    echo "Creating Public Key Info PEM: ${RSA_PUBLIC_KEY_INFO_SIGNING_REQUEST_PEM}"
    openssl req -pubkey -inform PEM -in "${RSA_SIGNING_REQUEST_PEM}" | \
        sed -n '/BEGIN\ PUBLIC\ KEY/,/END\ PUBLIC\ KEY/p' > "${RSA_PUBLIC_KEY_INFO_SIGNING_REQUEST_PEM}"

    # converting PEM to DER
    echo "Converting PEM to DER: ${RSA_PUBLIC_KEY_INFO_SIGNING_REQUEST_DER}"
    openssl asn1parse -inform PEM -in "${RSA_PUBLIC_KEY_INFO_SIGNING_REQUEST_PEM}" -out "${RSA_PUBLIC_KEY_INFO_SIGNING_REQUEST_DER}"
    echo ""
done
echo ""

echo "Extracting the SubjectPublicKeyInfo from RSA Self-Signed Certificates:"
echo "--------------------------------------------"
# from self-signed public key
for RSA_SELF_SIGNED_PUBLIC_KEY_PEM in "${FOLDER}"/rsa_self-signed-public-key*.pem; do
    GUID=$(openssl x509 -noout -in "${RSA_SELF_SIGNED_PUBLIC_KEY_PEM}" -subject | awk -F 'CN=' '{print $2}')
    RSA_PUBLIC_KEY_INFO_SELF_SIGNED_PEM="${FOLDER}/rsa_public-key-info_via_self-signed_${GUID}.pem" 
    RSA_PUBLIC_KEY_INFO_SELF_SIGNED_DER="${FOLDER}/rsa_public-key-info_via_self-signed_${GUID}.der"  

    # creating PEM signed certificate
    echo "Creating Public Key Info PEM: ${RSA_PUBLIC_KEY_INFO_SELF_SIGNED_PEM}"
    openssl x509 -pubkey -inform PEM -in "${RSA_SELF_SIGNED_PUBLIC_KEY_PEM}" | \
        sed -n '/BEGIN\ PUBLIC\ KEY/,/END\ PUBLIC\ KEY/p' > "${RSA_PUBLIC_KEY_INFO_SELF_SIGNED_PEM}"

    # converting PEM to DER
    echo "Converting PEM to DER: ${RSA_PUBLIC_KEY_INFO_SELF_SIGNED_DER}"
    openssl asn1parse -inform PEM -in "${RSA_PUBLIC_KEY_INFO_SELF_SIGNED_PEM}" -out "${RSA_PUBLIC_KEY_INFO_SELF_SIGNED_DER}"
    echo ""
done
echo ""


Extracting the SubjectPublicKeyInfo from RSA Signing Requests:
--------------------------------------------
Creating Public Key Info PEM: data/poc/rsa_public-key-info_via_signing-request_1beba118-c83f-4095-883d-3f19af5d58ca.pem
Converting PEM to DER: data/poc/rsa_public-key-info_via_signing-request_1beba118-c83f-4095-883d-3f19af5d58ca.der
    0:d=0  hl=4 l= 290 cons: SEQUENCE          
    4:d=1  hl=2 l=  13 cons: SEQUENCE          
    6:d=2  hl=2 l=   9 prim: OBJECT            :rsaEncryption
   17:d=2  hl=2 l=   0 prim: NULL              
   19:d=1  hl=4 l= 271 prim: BIT STRING        

Creating Public Key Info PEM: data/poc/rsa_public-key-info_via_signing-request_1eb88898-fb38-4a66-81a4-16e11175edb5.pem
Converting PEM to DER: data/poc/rsa_public-key-info_via_signing-request_1eb88898-fb38-4a66-81a4-16e11175edb5.der
    0:d=0  hl=4 l= 290 cons: SEQUENCE          
    4:d=1  hl=2 l=  13 cons: SEQUENCE          
    6:d=2  hl=2 l=   9 prim: OBJECT            :rsaEncryption
   17:d=2  hl

### Compare PublicKeyInfo Digests

In summary, we've performed the following actions:
1. Derived the public key from a single private key 
<br>(see `*public-key-info_via_private_key*.der`).
2. Extracted the public key material from multiple certificate signing requests
<br>(see `*public-key-info_via_signing-request*.der`).
3. Extracted the public key material from multiple self-signed certificates 
<br>(see `*public-key-info_via_self-signed*.der`).

Do we have message digests which match across all these different exports?

> Note 1: If you do not see the results you expect, make sure you are [Erase All RSA Key Data](#Erase-All-RSA-Key-Data) between test runs.

If you want more samples, then follow these steps without erasing or resetting the RSA data.
1. Run [Generate RSA Signing Requests](#generate-rsa-signing-requests) multiple times.  e.g. 4 more times for a total of 25 signing requests.
2. Run [Self-Sign RSA Signing Requests](#self-sign-rsa-signing-requests) just once.  
3. Run [Extract RSA SubjectPublicKeyInfo](#extract-rsa-subjectpublickeyinfo) just once.
4. Run this cell again.

In [27]:
%%bash
FOLDER="data/poc"

# basic error checking
if ! find "${FOLDER}" -type f -name "rsa_self-signed-public-key*.der" | grep -q .; then
    echo "WARNING: No RSA Self-Signed Certificates found."
    echo "         Please run the previous cells to generate RSA content."
    exit
fi

UNIQUE_DER_DIGEST_COUNT=$(sha256sum "${FOLDER}"/rsa_self-signed-public-key*.der \
    | awk '{print $1}' | sort | uniq | wc -l | awk '{print $1}')

UNIQUE_SPKI_DIGEST_COUNT=$(sha256sum "${FOLDER}"/rsa_public-key-info*.der \
    | awk '{print $1}' | sort | uniq | wc -l | awk '{print $1}')

# if UNIQUE_SPKI_DIGEST_COUNT greater than 1 then issue a warning
if [ "${UNIQUE_SPKI_DIGEST_COUNT}" -gt 1 ]; then
    echo "WARNING: Multiple SubjectPublicKeyInfo Digests found."
    echo "         Did you forget to erase all key data between runs?"
    echo ""
fi

echo "Summary of Unique RSA Key Data:"
echo "   Signed Certificates: ${UNIQUE_DER_DIGEST_COUNT}"
echo "  SubjectPublicKeyInfo: ${UNIQUE_SPKI_DIGEST_COUNT}"
echo ""

echo "Certificate (DER) Digests (SHA256):"
sha256sum "${FOLDER}"/rsa_self-signed-public-key*.der
echo ""

echo "SubjectPublicKeyInfo Digests (SHA256):"
echo "[SubjectPublicKeyInfo derived from the Private Key]"
sha256sum "${FOLDER}"/rsa_public-key-info_via_private-key*.der | sort
echo "[SubjectPublicKeyInfo extracted from Signing Requests]"
sha256sum "${FOLDER}"/rsa_public-key-info_via_signing-request*.der | sort
echo "[SubjectPublicKeyInfo extracted from Self-Signed Certificates]"
sha256sum "${FOLDER}"/rsa_public-key-info_via_self-signed*.der | sort


Summary of Unique RSA Key Data:
   Signed Certificates: 5
  SubjectPublicKeyInfo: 1

Certificate (DER) Digests (SHA256):
4811c1e17263ba7f1dcb65f52972ac95b35520b2bd64dbd44be21b637b90986b  data/poc/rsa_self-signed-public-key_1beba118-c83f-4095-883d-3f19af5d58ca.der
23a335a574495c6fd020ff7133298d2970afc5c8a6adf89cbd1edd43c70507b7  data/poc/rsa_self-signed-public-key_1eb88898-fb38-4a66-81a4-16e11175edb5.der
9ec052c09e00b348c114f8005980c822816dc5dc822c3f5cb28ce0729644c27c  data/poc/rsa_self-signed-public-key_1f181c3b-551e-4e63-85ac-0724124888ed.der
256aa394682a6054b2df337cbdc20d429faccacce52ee20889dcf8468283c18c  data/poc/rsa_self-signed-public-key_311affb2-a2e2-47de-88ee-5eb5cd0b5f59.der
50b03d4645e6e98820745ca9dbf6c89e212c00cbd1c165c895e4a93b04509ca0  data/poc/rsa_self-signed-public-key_3550c8ce-7c71-4570-8358-672787735503.der

SubjectPublicKeyInfo Digests (SHA256):
[SubjectPublicKeyInfo derived from the Private Key]
7739a822aa5745263aa46d9aa663b53b3c011f4ac552b548d80b4d7a669ccebd  data/p

### Erase All RSA Key Data

Use this to reset the proof-of-concept data related to RSA key data.

In [ ]:
%%bash
FOLDER="data/poc"

# if folder exists, remove it
rm -rf "${FOLDER}"/rsa_*

## EC

Resources:

1. [openssl-asn1parse](https://docs.openssl.org/master/man1/openssl-asn1parse/) processing ASN.1 files
2. [openssl-ec](https://docs.openssl.org/master/man1/openssl-ec/) generating elliptic curve private keys
3. [openssl-pkey](https://docs.openssl.org/master/man1/openssl-pkey/) interacting with private keys
4. [openssl-req](https://docs.openssl.org/master/man1/openssl-req/) processing signing-requests
5. [openssl-x509](https://docs.openssl.org/master/man1/openssl-x509/) processing X.509 certificates


### Generate EC Private Key

For illustration purposes, let us create a single RSA private key. 

> Warning: Please do not use these keys in any production or live environment as they are intended for demonstration purposes only.  As a _demo_, you have no control over who may have viewed or obtain a copy of this key material making these private keys potentially _public_.

In [28]:
%%bash
# preamble
FOLDER="data/poc"
mkdir -p "${FOLDER}"
EC_PRIVATE_KEY_PEM="${FOLDER}/ec_private-key.pem"
EC_PRIVATE_KEY_DER="${FOLDER}/ec_private-key.der"
EC_CURVE="prime256v1" # `prime256v1` and `secp384r1`

# validate environment
echo "Environment:"
echo "  Working Folder:  '${FOLDER}'"
echo "  Private Key PEM: '${EC_PRIVATE_KEY_PEM}'"
echo "  Private Key DER: '${EC_PRIVATE_KEY_DER}'"
echo "  Elliptic Curve:  '${EC_CURVE}'"
echo ""

# execute
echo "Generating EC Private Key:"

# create the private key in PEM form.
openssl ecparam -genkey -name "${EC_CURVE}" -noout -out "${EC_PRIVATE_KEY_PEM}"

# convert private key from PEM (ASCII) to DER (binary)
openssl ec -inform pem -in "${EC_PRIVATE_KEY_PEM}" -outform der -out "${EC_PRIVATE_KEY_DER}"
echo ""

# valdiate result
echo "Files:"
ls -l -go "${FOLDER}"/ec_private-key.*
echo ""

echo "File Types:"
file "${FOLDER}"/ec_private-key.*
echo ""

echo "File Digests (SHA256):"
sha256sum "${FOLDER}"/ec_private-key.*


Environment:
  Working Folder:  'data/poc'
  Private Key PEM: 'data/poc/ec_private-key.pem'
  Private Key DER: 'data/poc/ec_private-key.der'
  Elliptic Curve:  'prime256v1'

Generating EC Private Key:


read EC key
writing EC key



Files:
-rw------- 1 121 Nov 21 17:26 data/poc/ec_private-key.der
-rw------- 1 227 Nov 21 17:26 data/poc/ec_private-key.pem

File Types:
data/poc/ec_private-key.der: data
data/poc/ec_private-key.pem: PEM EC private key

File Digests (SHA256):
c1b2996955ed1f583dc7214cf409b130f9c261037e603605e0be6480561823af  data/poc/ec_private-key.der
df587fe6ea5b4dadec6f506da29f1895ef38d166fba521d5e4d47bdce5ec2d05  data/poc/ec_private-key.pem


### Derive the EC Public Key

The raw private key material itself does not contain the public key.  However we can derive the cryptographic public key material from the private key using one of the `openssl` command options.  
1. `openssl ec -pubout ...`
2. `openssl pkey -pubout ...`

In [29]:
%%bash
# preamble
FOLDER="data/poc"
EC_PRIVATE_KEY_PEM="${FOLDER}/ec_private-key.pem"
EC_PRIVATE_KEY_DER="${FOLDER}/ec_private-key.der"

# validate environment
echo "Environment:"
echo "  EC Private Key PEM: '${EC_PRIVATE_KEY_PEM}'"
echo "  EC Private Key DER: '${EC_PRIVATE_KEY_DER}'"
echo ""

# execute
echo "Deriving the EC Public Key using the corresponding Private Key (ec):"
openssl ec -pubout -inform PEM -in "${EC_PRIVATE_KEY_PEM}" \
    -outform PEM -out "${FOLDER}/ec_public-key-info_via_private-key_ec.pem"
openssl ec -pubout -inform DER -in "${EC_PRIVATE_KEY_DER}" \
    -outform DER -out "${FOLDER}/ec_public-key-info_via_private-key_ec.der"
echo ""

echo "Deriving the EC Public Key using the corresponding Private Key (pkey):"
openssl pkey -pubout -inform PEM -in "${EC_PRIVATE_KEY_PEM}" \
    -outform PEM -out "${FOLDER}"/ec_public-key-info_via_private-key_pkey.pem
openssl pkey -pubout -inform DER -in "${EC_PRIVATE_KEY_DER}" \
    -outform DER -out "${FOLDER}"/ec_public-key-info_via_private-key_pkey.der
echo ""

# validate results
echo "EC Files:"
ls -l -go "${FOLDER}"/ec_public-key-info_via_private-key*
echo ""

echo "EC File Types:"
file "${FOLDER}"/ec_public-key-info_via_private-key*
echo ""

echo "EC File Digests (SHA256):"
sha256sum "${FOLDER}"/ec_public-key-info_via_private-key*.der
echo "---"
sha256sum "${FOLDER}"/ec_public-key-info_via_private-key*.pem

Environment:
  EC Private Key PEM: 'data/poc/ec_private-key.pem'
  EC Private Key DER: 'data/poc/ec_private-key.der'

Deriving the EC Public Key using the corresponding Private Key (ec):


read EC key
writing EC key
read EC key
writing EC key



Deriving the EC Public Key using the corresponding Private Key (pkey):

EC Files:
-rw-rw-r-- 1  91 Nov 21 17:27 data/poc/ec_public-key-info_via_private-key_ec.der
-rw-rw-r-- 1 178 Nov 21 17:27 data/poc/ec_public-key-info_via_private-key_ec.pem
-rw-rw-r-- 1  91 Nov 21 17:27 data/poc/ec_public-key-info_via_private-key_pkey.der
-rw-rw-r-- 1 178 Nov 21 17:27 data/poc/ec_public-key-info_via_private-key_pkey.pem

EC File Types:
data/poc/ec_public-key-info_via_private-key_ec.der:   data
data/poc/ec_public-key-info_via_private-key_ec.pem:   ASCII text
data/poc/ec_public-key-info_via_private-key_pkey.der: data
data/poc/ec_public-key-info_via_private-key_pkey.pem: ASCII text

EC File Digests (SHA256):
7ee26e75f8ed2336375ee640ec4979fcf34dbc33c03b31e33ff7ef47b372e4d9  data/poc/ec_public-key-info_via_private-key_ec.der
7ee26e75f8ed2336375ee640ec4979fcf34dbc33c03b31e33ff7ef47b372e4d9  data/poc/ec_public-key-info_via_private-key_pkey.der
---
512bc157b8cba1308072b0fd6fdf2b4fa6858952f1494f72ce21470a3d

### Generate EC Signing Requests

Our goal is to create multiple unique signed public certificates each based on the single private key we just created.  To begin, we must create a set of _certificate signing requests_, each provided with a unique name.

To uniquely identify each signing request (and later the certificate) we apply a unique ID to the request `CommonName` field of the `Subject`.

In [30]:
%%bash
# preamble
FOLDER="data/poc"
EC_PRIVATE_KEY_PEM="${FOLDER}/ec_private-key.pem"
MAX_SIGNING_REQUESTS=5


echo "Environment:"
echo "  Signing Request Count: ${MAX_SIGNING_REQUESTS}"
echo ""

for ((i=1; i<="${MAX_SIGNING_REQUESTS}"; i++)); do
    GUID=$(uuidgen)
    EC_SIGNING_REQUEST_PEM="${FOLDER}/ec_signing-request_${GUID}.pem"
    EC_SIGNING_REQUEST_DER="${FOLDER}/ec_signing-request_${GUID}.der"
    
    # creates PEM signing request
    echo "Creating PEM: ${EC_SIGNING_REQUEST_PEM}"
    openssl req -new -sha256 -inform pem -key "${EC_PRIVATE_KEY_PEM}" \
      -outform pem -out "${EC_SIGNING_REQUEST_PEM}" -subj "/O=WARNING\: PRIVATE KEY MADE PUBLIC/CN=${GUID}"

    # converts PEM to DER signing request
    echo "Creating DER: ${EC_SIGNING_REQUEST_DER}"
    openssl req -inform PEM -in "${EC_SIGNING_REQUEST_PEM}" \
      -outform DER -out "${EC_SIGNING_REQUEST_DER}"

done
echo ""

echo "EC Signing Requests:"
ls -l -go "${FOLDER}"/ec_signing-request*
echo ""

echo "EC File Types:"
file "${FOLDER}"/ec_signing-request*
echo ""

echo "EC Signing Request Digests (SHA256):"
sha256sum "${FOLDER}"/ec_signing-request*.der
echo "---"
sha256sum "${FOLDER}"/ec_signing-request*.pem
echo ""


Environment:
  Signing Request Count: 5

Creating PEM: data/poc/ec_signing-request_c1076872-8c69-4e53-a1a3-e9e9d49b1b07.pem
Creating DER: data/poc/ec_signing-request_c1076872-8c69-4e53-a1a3-e9e9d49b1b07.der
Creating PEM: data/poc/ec_signing-request_36aeaaa4-97e1-425e-ab1c-a8b285aa089b.pem
Creating DER: data/poc/ec_signing-request_36aeaaa4-97e1-425e-ab1c-a8b285aa089b.der
Creating PEM: data/poc/ec_signing-request_f3613886-66c7-4e04-bfc5-32a4e0b73f84.pem
Creating DER: data/poc/ec_signing-request_f3613886-66c7-4e04-bfc5-32a4e0b73f84.der
Creating PEM: data/poc/ec_signing-request_606de235-eb43-4df2-9041-62114d8596d5.pem
Creating DER: data/poc/ec_signing-request_606de235-eb43-4df2-9041-62114d8596d5.der
Creating PEM: data/poc/ec_signing-request_fade2dc6-517a-413d-902e-59e4483dfa39.pem
Creating DER: data/poc/ec_signing-request_fade2dc6-517a-413d-902e-59e4483dfa39.der

EC Signing Requests:
-rw-rw-r-- 1 282 Nov 21 17:27 data/poc/ec_signing-request_36aeaaa4-97e1-425e-ab1c-a8b285aa089b.der
-rw-rw-r

### Self-Sign EC Signing Requests

Using the single private key, we can self-sign each of the signing requests to create a completed signed certificate for analysis.

In [31]:
%%bash
# preamble
FOLDER="data/poc"
EC_PRIVATE_KEY_PEM="${FOLDER}/ec_private-key.pem"
VALIDITY_DURATION=30
VALIDITY_INCREMENT=10

for EC_SIGNING_REQUEST_PEM in "${FOLDER}"/ec_signing-request*.pem; do
    VALIDITY_DURATION=$((VALIDITY_DURATION+VALIDITY_INCREMENT))
    GUID=$(openssl req -noout -in "${EC_SIGNING_REQUEST_PEM}" -subject | awk -F 'CN=' '{print $2}')
    EC_SELF_SIGNED_PUBLIC_KEY_PEM="${FOLDER}/ec_self-signed-public-key_${GUID}.pem" 
    EC_SELF_SIGNED_PUBLIC_KEY_DER="${FOLDER}/ec_self-signed-public-key_${GUID}.der"  

    # creating PEM signed certificate
    echo "Creating Self-Signed PEM: ${EC_SELF_SIGNED_PUBLIC_KEY_PEM}"
    openssl x509 -req -sha256 -days ${VALIDITY_DURATION} \
        -keyform PEM -signkey "${EC_PRIVATE_KEY_PEM}" \
        -inform PEM -in "${EC_SIGNING_REQUEST_PEM}" \
        -outform PEM -out "${EC_SELF_SIGNED_PUBLIC_KEY_PEM}"

    # converting PEM to DER
    echo "Converting PEM to DER: ${EC_SELF_SIGNED_PUBLIC_KEY_DER}"
    openssl x509 -inform PEM -in "${EC_SELF_SIGNED_PUBLIC_KEY_PEM}" \
        -outform DER -out "${EC_SELF_SIGNED_PUBLIC_KEY_DER}"
done
echo ""

echo "EC Self-Signed Certificates:"
ls -l -go "${FOLDER}"/ec_self-signed-public-key*
echo ""

echo "EC Self-Signed Certificate File Types:"
file "${FOLDER}"/ec_self-signed-public-key*
echo ""

echo "EC Self-Signed Certificate SHA256 Digests:"
sha256sum "${FOLDER}"/ec_self-signed-public-key*.der
echo "---"
sha256sum "${FOLDER}"/ec_self-signed-public-key*.pem
echo ""


Creating Self-Signed PEM: data/poc/ec_self-signed-public-key_36aeaaa4-97e1-425e-ab1c-a8b285aa089b.pem


Certificate request self-signature ok
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=36aeaaa4-97e1-425e-ab1c-a8b285aa089b


Converting PEM to DER: data/poc/ec_self-signed-public-key_36aeaaa4-97e1-425e-ab1c-a8b285aa089b.der
Creating Self-Signed PEM: data/poc/ec_self-signed-public-key_606de235-eb43-4df2-9041-62114d8596d5.pem


Certificate request self-signature ok
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=606de235-eb43-4df2-9041-62114d8596d5


Converting PEM to DER: data/poc/ec_self-signed-public-key_606de235-eb43-4df2-9041-62114d8596d5.der
Creating Self-Signed PEM: data/poc/ec_self-signed-public-key_c1076872-8c69-4e53-a1a3-e9e9d49b1b07.pem


...

Converting PEM to DER: data/poc/ec_self-signed-public-key_c1076872-8c69-4e53-a1a3-e9e9d49b1b07.der
Creating Self-Signed PEM: data/poc/ec_self-signed-public-key_f3613886-66c7-4e04-bfc5-32a4e0b73f84.pem


Certificate request self-signature ok
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=f3613886-66c7-4e04-bfc5-32a4e0b73f84


Converting PEM to DER: data/poc/ec_self-signed-public-key_f3613886-66c7-4e04-bfc5-32a4e0b73f84.der
Creating Self-Signed PEM: data/poc/ec_self-signed-public-key_fade2dc6-517a-413d-902e-59e4483dfa39.pem


Certificate request self-signature ok
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=fade2dc6-517a-413d-902e-59e4483dfa39


Converting PEM to DER: data/poc/ec_self-signed-public-key_fade2dc6-517a-413d-902e-59e4483dfa39.der

EC Self-Signed Certificates:
-rw-rw-r-- 1 476 Nov 21 17:27 data/poc/ec_self-signed-public-key_36aeaaa4-97e1-425e-ab1c-a8b285aa089b.der
-rw-rw-r-- 1 700 Nov 21 17:27 data/poc/ec_self-signed-public-key_36aeaaa4-97e1-425e-ab1c-a8b285aa089b.pem
-rw-rw-r-- 1 474 Nov 21 17:27 data/poc/ec_self-signed-public-key_606de235-eb43-4df2-9041-62114d8596d5.der
-rw-rw-r-- 1 696 Nov 21 17:27 data/poc/ec_self-signed-public-key_606de235-eb43-4df2-9041-62114d8596d5.pem
-rw-rw-r-- 1 476 Nov 21 17:27 data/poc/ec_self-signed-public-key_c1076872-8c69-4e53-a1a3-e9e9d49b1b07.der
-rw-rw-r-- 1 700 Nov 21 17:27 data/poc/ec_self-signed-public-key_c1076872-8c69-4e53-a1a3-e9e9d49b1b07.pem
-rw-rw-r-- 1 475 Nov 21 17:27 data/poc/ec_self-signed-public-key_f3613886-66c7-4e04-bfc5-32a4e0b73f84.der
-rw-rw-r-- 1 700 Nov 21 17:27 data/poc/ec_self-signed-public-key_f3613886-66c7-4e04-bfc5-32a4e0b73f84.pem
-rw-rw-r-- 1 476 Nov 21

### Display Certificate Details (Briefly)

Confirming our certificates have unique names and validity periods.

In [32]:
%%bash
FOLDER="data/poc"

# from signed certificates
echo "Basic Certificate Details:"
for EC_SELF_SIGNED_PUBLIC_KEY_PEM in "${FOLDER}"/ec_self-signed-public-key*.pem; do
    echo "--------------------------------------------"
    openssl x509 -noout -in "${EC_SELF_SIGNED_PUBLIC_KEY_PEM}" \
        -subject -serial -startdate -enddate
done
echo ""

Basic Certificate Details:
--------------------------------------------
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=36aeaaa4-97e1-425e-ab1c-a8b285aa089b
serial=30A857F25D72E5EA2D4C09E6D2253B0499E1FF01
notBefore=Nov 21 22:27:08 2024 GMT
notAfter=Dec 31 22:27:08 2024 GMT
--------------------------------------------
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=606de235-eb43-4df2-9041-62114d8596d5
serial=34D3B2FD2E695AD07EB651147E2F605B61F241E6
notBefore=Nov 21 22:27:08 2024 GMT
notAfter=Jan 10 22:27:08 2025 GMT
--------------------------------------------
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=c1076872-8c69-4e53-a1a3-e9e9d49b1b07
serial=0DAC84D5EF19FD242672366D462F4CEBEEEFA269
notBefore=Nov 21 22:27:08 2024 GMT
notAfter=Jan 20 22:27:08 2025 GMT
--------------------------------------------
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=f3613886-66c7-4e04-bfc5-32a4e0b73f84
serial=52CE1A54983F523A0BD6C4169525A48A71473216
notBefore=Nov 21 22:27:08 2024 GMT
notAfter=Jan 30 22:27:0

### Extract EC SubjectPublicKeyInfo

We already derived the raw public key material earlier by accessing the private key for this information.  Here, we'll shift our focus to both the signing requests and self-signed certificates for analysis.

First, we'll access the signing requests to extract the `SubjectPublicKeyInfo` data and convert it to DER (binary) format.  Then we'll access each self-signed certificate to perform a similar action.

> Note: when using the `-pubkey` option, the output will often contain two sections, in ASCII format, defining the `PUBLIC KEY` and `CERTIFICATE REQUEST`.  We use `sed` to parse out only the `PUBLIC KEY` portion of this output.

In [33]:
%%bash
FOLDER="data/poc"

echo "Extracting the SubjectPublicKeyInfo from EC Signing Requests:"
echo "--------------------------------------------"
# from signing request
for EC_SIGNING_REQUEST_PEM in "${FOLDER}"/ec_signing-request*.pem; do
    GUID=$(openssl req -noout -in "${EC_SIGNING_REQUEST_PEM}" -subject | awk -F 'CN=' '{print $2}')
    EC_PUBLIC_KEY_INFO_SIGNING_REQUEST_PEM="${FOLDER}/ec_public-key-info_via_signing-request_${GUID}.pem" 
    EC_PUBLIC_KEY_INFO_SIGNING_REQUEST_DER="${FOLDER}/ec_public-key-info_via_signing-request_${GUID}.der"  

    # creating PEM signed certificate
    echo "Creating Public Key Info PEM: ${EC_PUBLIC_KEY_INFO_SIGNING_REQUEST_PEM}"
    openssl req -pubkey -inform PEM -in "${EC_SIGNING_REQUEST_PEM}" | \
        sed -n '/BEGIN\ PUBLIC\ KEY/,/END\ PUBLIC\ KEY/p' > "${EC_PUBLIC_KEY_INFO_SIGNING_REQUEST_PEM}"

    # converting PEM to DER
    echo "Converting PEM to DER: ${EC_PUBLIC_KEY_INFO_SIGNING_REQUEST_DER}"
    openssl asn1parse -inform PEM -in "${EC_PUBLIC_KEY_INFO_SIGNING_REQUEST_PEM}" \
        -out "${EC_PUBLIC_KEY_INFO_SIGNING_REQUEST_DER}"
done
echo ""

echo "Extracting the SubjectPublicKeyInfo from EC Self-Signed Certificates:"
echo "--------------------------------------------"
# from self-signed public key
for EC_SELF_SIGNED_PUBLIC_KEY_PEM in "${FOLDER}"/ec_self-signed-public-key*.pem; do
    echo "Self-Signed Key: ${EC_SELF_SIGNED_PUBLIC_KEY_PEM}"
    GUID=$(openssl x509 -noout -in "${EC_SELF_SIGNED_PUBLIC_KEY_PEM}" -subject | awk -F 'CN=' '{print $2}')
    EC_PUBLIC_KEY_INFO_SELF_SIGNED_PEM="${FOLDER}/ec_public-key-info_via_self-signed_${GUID}.pem" 
    EC_PUBLIC_KEY_INFO_SELF_SIGNED_DER="${FOLDER}/ec_public-key-info_via_self-signed_${GUID}.der"  

    # creating PEM signed certificate
    echo "Creating Public Key Info PEM: ${EC_PUBLIC_KEY_INFO_SELF_SIGNED_PEM}"
    openssl x509 -pubkey -inform PEM -in "${EC_SELF_SIGNED_PUBLIC_KEY_PEM}" | \
        sed -n '/BEGIN\ PUBLIC\ KEY/,/END\ PUBLIC\ KEY/p' > "${EC_PUBLIC_KEY_INFO_SELF_SIGNED_PEM}"

    # converting PEM to DER
    echo "Converting PEM to DER: ${EC_PUBLIC_KEY_INFO_SELF_SIGNED_DER}"
    openssl asn1parse -inform PEM -in "${EC_PUBLIC_KEY_INFO_SELF_SIGNED_PEM}" \
        -out "${EC_PUBLIC_KEY_INFO_SELF_SIGNED_DER}"
done
echo ""

Extracting the SubjectPublicKeyInfo from EC Signing Requests:
--------------------------------------------
Creating Public Key Info PEM: data/poc/ec_public-key-info_via_signing-request_36aeaaa4-97e1-425e-ab1c-a8b285aa089b.pem
Converting PEM to DER: data/poc/ec_public-key-info_via_signing-request_36aeaaa4-97e1-425e-ab1c-a8b285aa089b.der
    0:d=0  hl=2 l=  89 cons: SEQUENCE          
    2:d=1  hl=2 l=  19 cons: SEQUENCE          
    4:d=2  hl=2 l=   7 prim: OBJECT            :id-ecPublicKey
   13:d=2  hl=2 l=   8 prim: OBJECT            :prime256v1
   23:d=1  hl=2 l=  66 prim: BIT STRING        
Creating Public Key Info PEM: data/poc/ec_public-key-info_via_signing-request_606de235-eb43-4df2-9041-62114d8596d5.pem
Converting PEM to DER: data/poc/ec_public-key-info_via_signing-request_606de235-eb43-4df2-9041-62114d8596d5.der
    0:d=0  hl=2 l=  89 cons: SEQUENCE          
    2:d=1  hl=2 l=  19 cons: SEQUENCE          
    4:d=2  hl=2 l=   7 prim: OBJECT            :id-ecPublicKey
   13:

### Compare PublicKeyInfo Digests

In summary, we've performed the following actions:
1. Derived the public key from a single private key 
<br>(see `ec_public-key-info_via_private_key*.der`).
2. Extracted the public key material from multiple certificate signing requests
<br>(see `ec_public-key-info_via_signing-request*.der`).
3. Extracted the public key material from multiple self-signed certificates 
<br>(see `ec_public-key-info_via_self-signed*.der`).

Do we have message digests which match across all these different exports?

> Note: If you do not see the results you expect, make sure you are [Erase All EC Key Data](#erase-all-ec-key-data)) between test runs.

If you want more samples, then follow these steps without erasing or resetting the EC data.
1. Run [Generate EC Signing Requests](#generate-ec-signing-requests) multiple times.  e.g. 4 more times for a total of 25 signing requests.
2. Run [Self-Sign EC Signing Requests](#self-sign-ec-signing-requests) just once.  
3. Run [Extract EC SubjectPublicKeyInfo](#extract-ec-subjectpublickeyinfo) just once.
4. Run this cell again.

In [34]:
%%bash
FOLDER="data/poc"

# basic error checking
if ! find "${FOLDER}" -type f -name "ec_self-signed-public-key*.der" | grep -q .; then
    echo "WARNING: No EC Self-Signed Certificates found."
    echo "         Please run the previous cells to generate EC content."
    exit
fi

UNIQUE_DER_DIGEST_COUNT=$(sha256sum "${FOLDER}"/ec_self-signed-public-key*.der \
    | awk '{print $1}' | sort | uniq | wc -l | awk '{print $1}')

UNIQUE_SPKI_DIGEST_COUNT=$(sha256sum "${FOLDER}"/ec_public-key-info*.der \
    | awk '{print $1}' | sort | uniq | wc -l | awk '{print $1}')

if [ "${UNIQUE_SPKI_DIGEST_COUNT}" -gt 1 ]; then
    echo "WARNING: Multiple SubjectPublicKeyInfo Digests found."
    echo "         Did you forget to erase all key data between runs?"
    echo ""
fi

echo "Summary of Unique EC Key Data:"
echo "   Signed Certificates: ${UNIQUE_DER_DIGEST_COUNT}"
echo "  SubjectPublicKeyInfo: ${UNIQUE_SPKI_DIGEST_COUNT}"
echo ""

echo "Certificate (DER) Digests (SHA256):"
sha256sum "${FOLDER}"/ec_self-signed-public-key*.der
echo ""

echo "SubjectPublicKeyInfo Digests (SHA256):"
echo "[SubjectPublicKeyInfo derived from the Private Key]"
sha256sum "${FOLDER}"/ec_public-key-info_via_private-key*.der | sort
echo "[SubjectPublicKeyInfo extracted from Signing Requests]"
sha256sum "${FOLDER}"/ec_public-key-info_via_signing-request*.der | sort
echo "[SubjectPublicKeyInfo extracted from Self-Signed Certificates]"
sha256sum "${FOLDER}"/ec_public-key-info_via_self-signed*.der | sort

Summary of Unique EC Key Data:
   Signed Certificates: 5
  SubjectPublicKeyInfo: 1

Certificate (DER) Digests (SHA256):
52792a4093ced52b245d959dc40757de5ce171fff9fb9b9d1d5e5828c1d6210c  data/poc/ec_self-signed-public-key_36aeaaa4-97e1-425e-ab1c-a8b285aa089b.der
907047576ba25bfd083a64e9fe1144bf8518a3b084341ec58e6c013200586414  data/poc/ec_self-signed-public-key_606de235-eb43-4df2-9041-62114d8596d5.der
7404856f4694d545096e21bb22be098f6ec235233c1ab96b462c4fe44a899f99  data/poc/ec_self-signed-public-key_c1076872-8c69-4e53-a1a3-e9e9d49b1b07.der
5c5b76c94c051c09ee616bd1ed0de09c1f5ba5dad3a1b2e5a1fbc09db8e9529d  data/poc/ec_self-signed-public-key_f3613886-66c7-4e04-bfc5-32a4e0b73f84.der
f8fb92b406ece38d864a236eb9f1cc62782f1f4c7fec1cb83207a97175754a54  data/poc/ec_self-signed-public-key_fade2dc6-517a-413d-902e-59e4483dfa39.der

SubjectPublicKeyInfo Digests (SHA256):
[SubjectPublicKeyInfo derived from the Private Key]
7ee26e75f8ed2336375ee640ec4979fcf34dbc33c03b31e33ff7ef47b372e4d9  data/poc/ec_

### Erase All EC Key Data

Use this to reset the proof-of-concept data related to EC key data.

In [ ]:
%%bash
FOLDER="data/poc"

# if folder exists, remove it
rm -rf "${FOLDER}"/ec_*

# Additional OpenSSL Commands and Data


## Cipher Suites

How to check cipher suite support, which should correlate with certificate type (`RSA` vs `ECDSA`).  List of available cipher suites from OpenSSL or OpenSSL compatible (e.g. LibreSSL).   Note, these are based on _your_ local OpenSSL installation.

In [35]:
%%bash
openssl ciphers -v

TLS_AES_256_GCM_SHA384         TLSv1.3 Kx=any      Au=any   Enc=AESGCM(256)            Mac=AEAD
TLS_CHACHA20_POLY1305_SHA256   TLSv1.3 Kx=any      Au=any   Enc=CHACHA20/POLY1305(256) Mac=AEAD
TLS_AES_128_GCM_SHA256         TLSv1.3 Kx=any      Au=any   Enc=AESGCM(128)            Mac=AEAD
ECDHE-ECDSA-AES256-GCM-SHA384  TLSv1.2 Kx=ECDH     Au=ECDSA Enc=AESGCM(256)            Mac=AEAD
ECDHE-RSA-AES256-GCM-SHA384    TLSv1.2 Kx=ECDH     Au=RSA   Enc=AESGCM(256)            Mac=AEAD
DHE-DSS-AES256-GCM-SHA384      TLSv1.2 Kx=DH       Au=DSS   Enc=AESGCM(256)            Mac=AEAD
DHE-RSA-AES256-GCM-SHA384      TLSv1.2 Kx=DH       Au=RSA   Enc=AESGCM(256)            Mac=AEAD
ECDHE-ECDSA-CHACHA20-POLY1305  TLSv1.2 Kx=ECDH     Au=ECDSA Enc=CHACHA20/POLY1305(256) Mac=AEAD
ECDHE-RSA-CHACHA20-POLY1305    TLSv1.2 Kx=ECDH     Au=RSA   Enc=CHACHA20/POLY1305(256) Mac=AEAD
DHE-RSA-CHACHA20-POLY1305      TLSv1.2 Kx=DH       Au=RSA   Enc=CHACHA20/POLY1305(256) Mac=AEAD
ECDHE-ECDSA-AES256-CCM         TLSv1.2 K

## OpenSSL Cipher Suites to JSON

In [36]:
import json

def openssl_cipher_suites_to_json(openssl_result:list) -> list:
    """Convers OpenSSL verbose cipher suite output to JSON.
    
    cipher_suites = [
        {
            "Cipher Suite": str,
            "Protocol": str,
            "Key Exchange": str,
            "Authentication": str,
            "Encryption": str,
            "MAC": str
        },
        {...}
    ]

    Args:
        openssl_result (list): list of lines from 'openssl ciphers -v'

    Returns:
        list: list of dictionaries
    """
    cipher_suites = []
    for line in openssl_result:
        fields = line.split()
        cipher_suite = {}
        cipher_suite['Cipher Suite'] = fields[0]
        cipher_suite['Protocol'] = fields[1]
        cipher_suite['Key Exchange'] = fields[2].split('=')[1]
        cipher_suite['Authentication'] = fields[3].split('=')[1]
        cipher_suite['Encryption'] = fields[4].split('=')[1]
        cipher_suite['MAC'] = fields[5].split('=')[1]

        cipher_suites.append(cipher_suite)
    return cipher_suites

# using a Jupyter 'feature'.  Caution executing external code or binaries from within your notebooks.
openssl_result = !openssl ciphers -v # use 'RSA' or 'AES' or other filter keywords

suites_json = openssl_cipher_suites_to_json(openssl_result)

print(json.dumps(suites_json, indent=2))


[
  {
    "Cipher Suite": "TLS_AES_256_GCM_SHA384",
    "Protocol": "TLSv1.3",
    "Key Exchange": "any",
    "Authentication": "any",
    "Encryption": "AESGCM(256)",
    "MAC": "AEAD"
  },
  {
    "Cipher Suite": "TLS_CHACHA20_POLY1305_SHA256",
    "Protocol": "TLSv1.3",
    "Key Exchange": "any",
    "Authentication": "any",
    "Encryption": "CHACHA20/POLY1305(256)",
    "MAC": "AEAD"
  },
  {
    "Cipher Suite": "TLS_AES_128_GCM_SHA256",
    "Protocol": "TLSv1.3",
    "Key Exchange": "any",
    "Authentication": "any",
    "Encryption": "AESGCM(128)",
    "MAC": "AEAD"
  },
  {
    "Cipher Suite": "ECDHE-ECDSA-AES256-GCM-SHA384",
    "Protocol": "TLSv1.2",
    "Key Exchange": "ECDH",
    "Authentication": "ECDSA",
    "Encryption": "AESGCM(256)",
    "MAC": "AEAD"
  },
  {
    "Cipher Suite": "ECDHE-RSA-AES256-GCM-SHA384",
    "Protocol": "TLSv1.2",
    "Key Exchange": "ECDH",
    "Authentication": "RSA",
    "Encryption": "AESGCM(256)",
    "MAC": "AEAD"
  },
  {
    "Cipher Suit

OpenSSL RSA Cipher Suites

We can query `openssl` for various cipher suites based on one or more cipher string values.  See [OpenSSL man page on Ciphers: Cipher Strings](https://docs.openssl.org/1.1.1/man1/ciphers/)

In [37]:
%%bash

# create an array of strings "RSA", "ECDH", "DHE", "ECDSA", "AES", "RSA"
CIPHERS=("ECDH" "DHE" "ECDSA" "AES" "RSA")

# iterate over the array with openssl ciphers -v
for CIPHER in "${CIPHERS[@]}"; do
    echo "Ciphers: '${CIPHER}'"
    openssl ciphers -V "${CIPHER}"
    echo ""
done


Ciphers: 'ECDH'
          0x13,0x02 - TLS_AES_256_GCM_SHA384         TLSv1.3 Kx=any      Au=any   Enc=AESGCM(256)            Mac=AEAD
          0x13,0x03 - TLS_CHACHA20_POLY1305_SHA256   TLSv1.3 Kx=any      Au=any   Enc=CHACHA20/POLY1305(256) Mac=AEAD
          0x13,0x01 - TLS_AES_128_GCM_SHA256         TLSv1.3 Kx=any      Au=any   Enc=AESGCM(128)            Mac=AEAD
          0xC0,0x2C - ECDHE-ECDSA-AES256-GCM-SHA384  TLSv1.2 Kx=ECDH     Au=ECDSA Enc=AESGCM(256)            Mac=AEAD
          0xC0,0x30 - ECDHE-RSA-AES256-GCM-SHA384    TLSv1.2 Kx=ECDH     Au=RSA   Enc=AESGCM(256)            Mac=AEAD
          0xCC,0xA9 - ECDHE-ECDSA-CHACHA20-POLY1305  TLSv1.2 Kx=ECDH     Au=ECDSA Enc=CHACHA20/POLY1305(256) Mac=AEAD
          0xCC,0xA8 - ECDHE-RSA-CHACHA20-POLY1305    TLSv1.2 Kx=ECDH     Au=RSA   Enc=CHACHA20/POLY1305(256) Mac=AEAD
          0xC0,0xAD - ECDHE-ECDSA-AES256-CCM         TLSv1.2 Kx=ECDH     Au=ECDSA Enc=AESCCM(256)            Mac=AEAD
          0xC0,0x5D - ECDHE-ECDSA-ARIA25

Using `openssl` to connect to a web site and display the certificate in an on-demand fashion.

> Note: Web sites can be load balanced across many IPv4 and IPv6 addresses.  This example merely accesses the first available.

In [38]:
#!/bin/bash

#cipher_suite="\'ECDHE-ECDSA-AES128-SHA\'" # RSA key type
# cipher_suite='ECDHE-ECDSA-AES256-GCM-SHA384' # EC key type

! echo "[ OpenSSL - Connection Details ]============================"
! echo | openssl s_client -showcerts -cipher "ECDHE-ECDSA-AES128-SHA" -servername www.google.com -connect www.google.com:443 2>&1
! echo ""
! echo "[ OpenSSL - Certificate Details ]==========================="
! echo | openssl s_client -showcerts -cipher 'ECDHE-ECDSA-AES128-SHA' -servername www.google.com -connect www.google.com:443 2>&1 | openssl x509 -noout -text


[ OpenSSL - Connection Details ]============================
Connecting to 142.250.191.228
CONNECTED(00000003)
depth=2 C=US, O=Google Trust Services LLC, CN=GTS Root R1
verify return:1
depth=1 C=US, O=Google Trust Services, CN=WR2
verify return:1
depth=0 CN=www.google.com
verify return:1
---
Certificate chain
 0 s:CN=www.google.com
   i:C=US, O=Google Trust Services, CN=WR2
   a:PKEY: id-ecPublicKey, 256 (bit); sigalg: RSA-SHA256
   v:NotBefore: Oct 21 08:38:45 2024 GMT; NotAfter: Jan 13 08:38:44 2025 GMT
-----BEGIN CERTIFICATE-----
MIIEWTCCA0GgAwIBAgIRAJ2cAzVYnqNCEHqwkZPYO2IwDQYJKoZIhvcNAQELBQAw
OzELMAkGA1UEBhMCVVMxHjAcBgNVBAoTFUdvb2dsZSBUcnVzdCBTZXJ2aWNlczEM
MAoGA1UEAxMDV1IyMB4XDTI0MTAyMTA4Mzg0NVoXDTI1MDExMzA4Mzg0NFowGTEX
MBUGA1UEAxMOd3d3Lmdvb2dsZS5jb20wWTATBgcqhkjOPQIBBggqhkjOPQMBBwNC
AAReIlhtyqRSkOSub0oJZ+urCnuQCfHorA21lF6ZwayGaorSZE6eI9ddnffWdi/N
qtni9oRSHO7NiQ1dEQQr8sWMo4ICQzCCAj8wDgYDVR0PAQH/BAQDAgeAMBMGA1Ud
JQQMMAoGCCsGAQUFBwMBMAwGA1UdEwEB/wQCMAAwHQYDVR0OBBYEFMNA0KyX4PFE
qhcU7a